**Введите ваше ФИО**

In [ ]:
print('Тыщенко Илья')

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение объектов, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests
from PIL import Image
from IPython.display import display
import io
import time


def apod_day():
  while True:
    url = 'https://api.nasa.gov/planetary/apod?api_key=DEMO_KEY'
    response = requests.get(url)
    if response.status_code == 200:
      apod_url = response.json()['url']
    response_img = requests.get(apod_url)
    img = Image.open(io.BytesIO(response_img.content))
    display(img)
    print('Explanation:', response.json()['explanation'])
    print('''
Для повторного использования функции нажмите "ENTER"
Для возвращения в главное меню введите "BACK"''')
    command = input() # Я не знаю почему но этот ввод просто не появляется в консоли, возможно его перекрывает как-то картинка. Не знаю как это решить, но если не отображать картинку он там есть и хорошо работает :)
    if command == 'BACK':
      break


def mars_rover_photos():
  while True:

    while True:
      n = 0
      date = input('Введите дату в формате(ГГГГ-ММ-ДД): ')
      try:
        valid_date = time.strptime(date, '%Y-%m-%d')
        n = 1
      except ValueError:
        print('Дата введена неверно! Попробуйте ещё раз!')
      if n > 0: break
    print('''
    ================================================================
    FHAZ - Передняя камера
    RHAZ -Задняя камера
    MAST - Мачтовая камера
    CHEMCAM - Химико-камерный комплекс
    MAHLI - Марсианский сканер с ручным объективом
    MARDI - Камера спуска с Марса
    NAVCAM - Навигационная камера
    PANCAM - Панорамная камера
    MINITES - Миниатюрный термоэмиссионный спектрометр (Мини-ТЭС)
    ================================================================
  ''')
    camera = input('Выберите тип камеры: ')
    url = f'https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos'
    params = {
        'earth_date': date,
        'camera': camera,
        'api_key': 'DEMO_KEY'
    }
    response = requests.get(url, params = params)
    if response.status_code == 200:
      if not response.json()["photos"]:
        print('Данные по данной дате отсутствуют!')
      else:
        img_url = response.json()['photos'][0]['img_src']
        response_img = requests.get(img_url)
        img = Image.open(io.BytesIO(response_img.content))
        display(img)
    else:
      print('Ошибка!')
    print('''
Для повторного использования функции нажмите "ENTER"
Для возвращения в главное меню введите "BACK"''')
    command = input() # Тут тоже картинка закрывает этот ввод
    if command == 'BACK':
      break

def asteroids_find():
  while True:
    while True:
      date = input('Введите дату в формате(ГГГГ-ММ-ДД): ')
      try:
        valid_date = time.strptime(date, '%Y-%m-%d')
        break
      except ValueError:
        print('Дата введена неверно! Попробуйте ещё раз!')
    url = f'https://api.nasa.gov/neo/rest/v1/feed'
    params = {
        'start_date': date,
        'end_date': date,
        'api_key': 'DEMO_KEY'
    }
    response = requests.get(url, params = params)
    if response.status_code == 200:
      if not response.json()["near_earth_objects"]:
        print('Данные по данной дате отсутствуют!')
      else:
        print('=====================================================================')
        print('              Объекты сближающиеся с Землей', date)
        print('=====================================================================')
        for i in range(0,response.json()["element_count"] ):
          print(response.json()["near_earth_objects"][date][i]["name"])
          print('Абсолютная величина (h): ', response.json()["near_earth_objects"][date][i]["absolute_magnitude_h"])
          print('Минимальный диаметр (km): ', response.json()["near_earth_objects"][date][i]["estimated_diameter"]['kilometers']['estimated_diameter_min'])
          print('Максимальный диаметр (km): ', response.json()["near_earth_objects"][date][i]["estimated_diameter"]['kilometers']['estimated_diameter_max'])
          if response.json()["near_earth_objects"][date][i]["is_potentially_hazardous_asteroid"] == False:
            print('Астероид не опасен!')
          else:
            print('Астероид опасен!')
          print('=====================================================================')
    else:
      print('Ошибка!')
    print('''
Для повторного использования функции нажмите "ENTER"
Для возвращения в главное меню введите "BACK"''')
    command = input()
    if command == 'BACK':
      break

while True:
  print('Выберите одну из функций:')
  print('''
  ============================================
  1 - Астрономическая картинка дня
  2 - Фотографии с марсохода
  3 - Объекты, сближающиеся с Землей (ОСЗ)
  4 - Завершить работу
  ============================================''')
  choice = int(input())
  if choice == 1:
    apod_day()
  elif choice == 2:
    mars_rover_photos()
  elif choice == 3:
    asteroids_find()
  elif choice == 4:
    break
  else:
    print('Функция выбрана неверно! Попробуйте ещё раз!')

# Я не сделал о космической погоде потому что я не понял какие данные брать от туда, извините


# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


# Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код